In [1]:
import numpy as np
from cvxopt import matrix
from cvxopt.blas import dot
from cvxopt.solvers import qp
import pickle
from sklearn.linear_model import LinearRegression
from sklearn import decomposition
import pandas as pd
from numpy import linalg
import math
from sklearn.covariance import LedoitWolf
import cvxopt

In [2]:
def load_object(file_name):
    "load the pickled object"
    with open(file_name, 'rb') as f:
        return pickle.load(f)


def view_data(data_path):
    data = load_object(data_path)
    prices = data['prices']
    names = data['features']['names']
    features = data['features']['values']
    print(prices.shape)
    print(names)
    print(features.shape)
    return prices, features, names

# Arbitrary Stock Iteration

In [3]:
#Say we are given some new price/factors vector
prices, features, names = view_data('C3_train.pkl')

#Just take the first 500 features and the 501 prices, Just like train data



#Log the price vector
df = pd.DataFrame(prices)
returns_df = df.pct_change()
returns_df.drop(0, axis = 0, inplace = True)



(757, 680)
['labour cost', 'analyst projected total earnings', 'weighted average outstanding shares', 'R&D intensity index', 'relative strength index', 'total assets', 'net book value', 'analyst sentiment', 'market share', 'Aggregate Capital Cost']
(756, 680, 10)


In [4]:
q = []
print((features.shape))
for k in range(0,len(prices[-1])):    
    #S0 to F0
    pca = decomposition.PCA(n_components = 3)
    model = LinearRegression()

    s0 = returns_df.iloc[-51:-2,k] 
    f0 = features[:,k].copy()
    
    f0 = f0[-52:-2]
    df = pd.DataFrame(f0)
    featureschange_df = df.pct_change()
    f0 = featureschange_df.drop(0, axis = 0)
    f0.fillna(0, inplace = True)


    #PCA on 5 components
    f0 = pca.fit_transform(f0)


    #Fit
    reg1 = model.fit(f0, s0)
    coef = reg1.coef_

    s1 = (prices[-1][k]  - prices[-2][k]) / prices[-2][k] 
    predicted_f1 = coef.T * s1


    ######################################################################


    # F0 to S1

    s1 = returns_df.iloc[-50:-1,k] 

    f0_sqr = np.square(f0)
    f0 = np.concatenate([f0, f0_sqr], axis = 1)


    reg2 = model.fit(f0, s1)
    predicted_f1
    predicted_f1squared = np.square(predicted_f1)
    predicted_f1 = np.concatenate([predicted_f1, predicted_f1squared])

    x = reg2.predict(predicted_f1.reshape(1,-1))
    q.append(x)


(756, 680, 10)


In [5]:
LedWolf = LedoitWolf().fit(prices)
cov = LedWolf.covariance_

In [6]:
cvxopt.solvers.options['show_progress'] = False

In [7]:
from math import sqrt
n = len(returns_df.T)
S = matrix(cov)
pbar = matrix(z)
G = matrix(0.0, (n,n))
G[::n+1] = -1.0
h = matrix(0.0, (n,1))
A = matrix(1.0, (1,n))
b = matrix(1.0)

# Compute trade-off.
N = 50
mus = [ 10**(5.0*t/N-1.0) for t in range(N) ]
#neg_mus = [ -10**(5.0*t/N-1.0) for t in range(N) ]
#mus = neg_mus + mus
portfolios = [ qp(mu*S, -pbar, G, h, A, b)['x'] for mu in mus ]
returns = [ dot(pbar,x) for x in portfolios ]
risks = [ sqrt(dot(x, S*x)) for x in portfolios ]

NameError: name 'z' is not defined

In [ ]:
sharpe = []
for x, y in zip(returns, risks):
    sharpe.append(x / np.square(y))
import operator
index, value = max(enumerate(sharpe), key=operator.itemgetter(1))
print(np.array(portfolios[index]))